In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_decision_forests as tfdf

ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [ ]:
# Load training data
train_df_ori = pd.read_csv('train.csv')

# Load test data

test_df_ori = pd.read_csv('test.csv')

In [114]:
train_df_ori.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [115]:
test_df_ori.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [116]:
train_df_ori['HomePlanet'].mode()

0    Earth
Name: HomePlanet, dtype: object

In [117]:
train_df_ori['HomePlanet'] = train_df_ori['HomePlanet'].fillna(train_df_ori['HomePlanet'].mode()[0])
train_df_ori['HomePlanet'] = train_df_ori['HomePlanet'].astype('category')
train_df_ori['CryoSleep'] = train_df_ori['CryoSleep'].fillna(train_df_ori['CryoSleep'].mode()[0])
train_df_ori['CryoSleep'] = train_df_ori['CryoSleep'].astype('category')
train_df_ori['Destination'] = train_df_ori['Destination'].fillna(train_df_ori['Destination'].mode()[0])
train_df_ori['Destination'] = train_df_ori['Destination'].astype('category')

train_df_ori["VIP"] = train_df_ori["VIP"].fillna(False).astype('category')

train_df_ori[['Deck', 'Num', 'Side']] = train_df_ori['Cabin'].str.split('/', expand=True)
train_df_ori['Deck'].fillna(0, inplace=True)
train_df_ori['Num'].fillna(0, inplace=True)
train_df_ori['Side'].fillna(0, inplace=True)

train_df_ori['Deck'] = train_df_ori['Deck'].astype('category')
train_df_ori['Side'] = train_df_ori['Side'].astype('category')
train_df_ori['Num'] = train_df_ori['Num'].astype('int')

train_df_ori.drop(columns=['Cabin', 'Name', 'PassengerId'], inplace=True)


C:\Users\user\AppData\Local\Temp\ipykernel_26800\1349147999.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df_ori['CryoSleep'] = train_df_ori['CryoSleep'].fillna(train_df_ori['CryoSleep'].mode()[0])
C:\Users\user\AppData\Local\Temp\ipykernel_26800\1349147999.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df_ori["VIP"] = train_df_ori["VIP"].fillna(False).astype('category')
C:\Users\user\AppData\Local\Temp\ipykernel_26800\1349147999.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained as

In [118]:
train_df_ori.describe(include='all')

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
count,8693,8693,8693,8514.000000,8693,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693,8693,8693.000000,8693
unique,3,2,3,NaN,2,NaN,NaN,NaN,NaN,NaN,2,9,NaN,3
top,Earth,False,TRAPPIST-1e,NaN,False,NaN,NaN,NaN,NaN,NaN,True,F,NaN,S
freq,4803,5656,6097,NaN,8494,NaN,NaN,NaN,NaN,NaN,4378,2794,NaN,4288
mean,NaN,NaN,NaN,28.827930,NaN,224.687617,458.077203,173.729169,311.138778,304.854791,NaN,NaN,586.624065,NaN
std,NaN,NaN,NaN,14.489021,NaN,666.717663,1611.489240,604.696458,1136.705535,1145.717189,NaN,NaN,513.880084,NaN
min,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,19.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,152.000000,NaN
50%,NaN,NaN,NaN,27.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,407.000000,NaN
75%,NaN,NaN,NaN,38.000000,NaN,47.000000,76.000000,27.000000,59.000000,46.000000,NaN,NaN,983.000000,NaN


In [119]:
categorical_col = train_df_ori.select_dtypes(exclude=np.number).columns.tolist()

for col in categorical_col:
    print(col)
    train_df_ori[col] = train_df_ori[col].astype('category')

HomePlanet
CryoSleep
Destination
VIP
Transported
Deck
Side


In [120]:
train_df_ori.dtypes

HomePlanet      category
CryoSleep       category
Destination     category
Age              float64
VIP             category
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
Transported     category
Deck            category
Num                int64
Side            category
dtype: object

In [121]:
x = train_df_ori.drop(columns = ["Transported"])
x

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,E,608,S


In [122]:
y = pd.DataFrame(train_df_ori["Transported"])
y

,Transported
0,False
1,True
2,False
3,False
4,True
...,...
8688,False
8689,False
8690,True
8691,False


In [133]:
train_df_ori["HomePlanet"].astype(int)

ValueError: Cannot cast object dtype to int64

In [123]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2)

# y_train = y_train['Transported'].astype(int)
# y_test = y_test['Transported'].astype(int)

# D_train = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
# D_test = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)


In [124]:
X_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num
count,6814.000000,6809.000000,6809.000000,6791.000000,6811.000000,6797.000000,6954.000000
mean,28.927062,224.564547,461.212366,176.116625,322.750991,307.292629,583.890423
std,14.457308,668.131903,1583.632660,614.216764,1182.967970,1127.995325,514.235549
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,153.250000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,397.000000
75%,38.000000,47.000000,81.000000,29.500000,61.000000,46.000000,981.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1894.000000


In [125]:
X_train.describe(exclude=['number'])

,HomePlanet,CryoSleep,Destination,VIP,Deck,Side
count,6954,6954,6954,6954,6954,6954
unique,3,2,3,2,9,3
top,Earth,False,TRAPPIST-1e,False,F,S
freq,3829,4554,4866,6802,2236,3446


In [126]:
X_test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num
count,1700.000000,1703.000000,1701.000000,1694.000000,1699.000000,1708.000000,1739.000000
mean,28.430588,225.179683,445.527337,164.158205,264.587404,295.153396,597.555492
std,14.612975,661.227736,1718.914694,564.993967,927.231715,1213.977492,512.457971
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,150.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,438.000000
75%,37.000000,45.000000,53.000000,18.750000,50.500000,45.250000,1000.500000
max,77.000000,9920.000000,26830.000000,12253.000000,13902.000000,20336.000000,1888.000000


In [127]:
y_train

,Transported
1691,True
1535,True
8476,True
3158,False
3368,True
...,...
5509,True
5612,False
7278,True
4085,True


In [128]:
y_train

,Transported
1691,True
1535,True
8476,True
3158,False
3368,True
...,...
5509,True
5612,False
7278,True
4085,True


In [131]:
X_train.dtypes

HomePlanet      category
CryoSleep       category
Destination     category
Age              float64
VIP             category
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
Deck            category
Num                int64
Side            category
dtype: object

In [129]:
# Convert y_train and y_test to proper format (1D array with numeric values)
y_train_fixed = y_train['Transported'].astype(int)
y_test_fixed = y_test['Transported'].astype(int)

# For binary classification, use 'binary:logistic' objective
model = xgb.XGBClassifier(
    objective="binary:logistic",  # Changed from multi:softprob
    enable_categorical=True, 
    random_state=42
)
model.fit(X_train, y_train_fixed)

TypeError: object of type 'int' has no len()

In [ ]:
model = xgb.XGBClassifier(objective="multi:softprob", num_class=2, enable_categorical=True, random_state=42)
model.fit(X_train, y_train)

TypeError: object of type 'numpy.bool' has no len()

In [ ]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [ ]:
preds

array([2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2,
       0, 2, 0, 2, 0, 1, 1, 1])